# Testing on PISA dataset with randomized missing data, substituting missing values with means of the item's values and calculating counterexamples with pairwise differences of item  correspondences.

1. Importing package and dataset

In [2]:
!pip install iita_python
!git clone https://gist.github.com/717f0147675b0c8ed25e50d583c943bf.git

import numpy as np
import iita_python as iita
import iita_python.fit_metrics as iita_fm
from iita_python.utils import read_rp
from iita_python.additional_ce import AdditionalCEDataset

from random import randint, choice, shuffle

fatal: destination path '717f0147675b0c8ed25e50d583c943bf' already exists and is not an empty directory.


2. Testing function

In [14]:
def test(metric, skips):
  correct = True
  correct_qo = None
  correct_count = 0
  data = read_rp('./717f0147675b0c8ed25e50d583c943bf/pisa.csv')

  while (correct and correct_count < data.shape[0]*data.shape[1] - skips * 2):
    print(correct_count)
    test_dataset = AdditionalCEDataset(data)
    unfolded_ce = iita.unfold_examples(test_dataset.missing_value_substitution_ce())
    qos = iita.ind_gen(unfolded_ce, test_dataset.items)

    best_qo_id = -1
    best_qo_diff = float('inf')
    for i, qo in enumerate(qos):
      qo_diff = metric(test_dataset, qo)
      if (qo_diff < best_qo_diff):
        best_qo_diff = qo_diff
        best_qo_id = i

    best_qo = sorted([(int(a), int(b)) for a, b in qos[best_qo_id].get_edge_list()])
    if (correct_qo is None):
      correct_qo = best_qo

    if (best_qo != correct_qo):
      correct = False
    else:
      correct_count += skips

      for _ in range(skips):
        while (True):
          a = randint(0, test_dataset.items - 1)
          b = randint(0, test_dataset.subjects - 1)
          # print(f'trying {a} {b}')
          # print(data.loc[b, a])
          if (not (np.isnan(data.loc[b, a]) or (np.nansum(data.to_numpy(), axis=0)[a] == 1))):
            break;
        data.loc[b, a] = np.nan

  return correct_count

In [15]:
def iter_test(metric, skips, iters, testf, **kwargs):
  iter_res = []
  for i in range(iters):
    print(f'ITER {i}')

    res = testf(metric, skips, **kwargs)
    print(res)
    iter_res.append(res)
  return iter_res

3. Running the tests

In [16]:
iters = 100 #amount of iterations to do
skips = 5 #amount of missing values to add at a time

res = iter_test(iita_fm.mini_iita_fit, skips, iters, test)

Streaming output truncated to the last 5000 lines.
180
185
190
195
200
205
210
215
220
225
230
235
240
245
250
255
260
265
270
275
280
285
290
295
300
305
310
315
320
325
330
335
340
345
350
355
360
365
370
375
380
385
390
395
400
405
410
415
420
425
430
430
ITER 34
0
5
10
15
20
25
30
35
40
45
50
55
60
65
70
75
80
85
90
95
100
105
110
115
120
125
130
135
140
145
150
155
160
165
170
175
180
185
190
195
200
205
210
215
220
220
ITER 35
0
5
10
15
20
25
30
35
40
45
50
55
60
65
70
75
80
85
90
95
100
105
110
115
120
125
130
135
140
145
150
155
160
165
170
175
180
185
190
195
200
205
210
215
220
225
230
235
240
245
250
255
260
265
270
275
280
285
290
295
300
305
310
315
320
325
330
335
340
345
350
355
360
365
370
375
380
385
390
395
400
405
405
ITER 36
0
5
10
15
20
25
30
35
40
45
50
55
60
65
70
75
80
85
90
95
100
105
110
115
120
125
130
135
140
145
150
155
160
165
170
175
180
185
190
195
200
205
210
215
220
225
230
235
240
245
250
255
260
265
270
275
280
285
290
295
300
305
310
315
320
325
330

In [19]:
print("[")
for i in range(10):
  print("    ", end="")
  for j in range(10):
    print(f"{res[10*i + j]}, ", end="")
  print()
print("]")

[
    230, 150, 270, 275, 610, 340, 370, 260, 100, 275, 
    630, 320, 300, 245, 245, 540, 485, 250, 330, 280, 
    215, 310, 295, 450, 385, 625, 505, 260, 305, 540, 
    495, 445, 535, 430, 220, 405, 355, 295, 345, 430, 
    675, 440, 525, 355, 375, 525, 415, 280, 255, 390, 
    350, 540, 395, 335, 305, 295, 425, 305, 235, 475, 
    385, 250, 300, 600, 445, 240, 485, 200, 210, 290, 
    380, 155, 385, 205, 195, 370, 315, 415, 510, 395, 
    255, 370, 520, 155, 515, 265, 240, 155, 305, 475, 
    280, 560, 170, 505, 600, 310, 245, 285, 505, 360, 
]


4. Analyzing the tests

Running the tests is quite a long process, so I did it for 100 iterations and saved the results in the next cell

Skip the next cell if you run the tests on your own

In [20]:
res = [
    230, 150, 270, 275, 610, 340, 370, 260, 100, 275,
    630, 320, 300, 245, 245, 540, 485, 250, 330, 280,
    215, 310, 295, 450, 385, 625, 505, 260, 305, 540,
    495, 445, 535, 430, 220, 405, 355, 295, 345, 430,
    675, 440, 525, 355, 375, 525, 415, 280, 255, 390,
    350, 540, 395, 335, 305, 295, 425, 305, 235, 475,
    385, 250, 300, 600, 445, 240, 485, 200, 210, 290,
    380, 155, 385, 205, 195, 370, 315, 415, 510, 395,
    255, 370, 520, 155, 515, 265, 240, 155, 305, 475,
    280, 560, 170, 505, 600, 310, 245, 285, 505, 360,
]

In [21]:
data = read_rp('./717f0147675b0c8ed25e50d583c943bf/pisa.csv')
res = (np.array(res) / (data.shape[0] * data.shape[1])).round(3)

4.1. Average

In [22]:
np.mean(res).round(3)

np.float64(0.212)

4.2. Standard deviation

In [23]:
np.std(res).round(3)

np.float64(0.074)

5. Testing function for biased item choice

In [24]:
def test_biased_items(metric, skips, bias):
  correct = True
  correct_qo = None
  correct_count = 0
  data = read_rp('./717f0147675b0c8ed25e50d583c943bf/pisa.csv')

  choicePool = []
  items = list(range(data.shape[1]))
  shuffle(items)

  for i, item in enumerate(items):
    for _ in range(bias[i]):
      choicePool.append(item)

  while (correct and correct_count < data.shape[0]*data.shape[1] - skips * 2):
    print(correct_count)
    test_dataset = AdditionalCEDataset(data)
    unfolded_ce = iita.unfold_examples(test_dataset.missing_value_substitution_ce())
    qos = iita.ind_gen(unfolded_ce, test_dataset.items)

    best_qo_id = -1
    best_qo_diff = float('inf')
    for i, qo in enumerate(qos):
      qo_diff = metric(test_dataset, qo)
      if (qo_diff < best_qo_diff):
        best_qo_diff = qo_diff
        best_qo_id = i

    best_qo = sorted([(int(a), int(b)) for a, b in qos[best_qo_id].get_edge_list()])
    if (correct_qo is None):
      correct_qo = best_qo

    if (best_qo != correct_qo):
      correct = False
    else:
      correct_count += skips

      for _ in range(skips):
        while (True):
          a = choice(choicePool)
          b = randint(0, test_dataset.subjects - 1)
          # print(f'trying {a} {b}')
          # print(data.loc[b, a])
          if (not (np.isnan(data.loc[b, a]) or (np.nansum(data.to_numpy(), axis=0)[a] == 1))):
            break;
        data.loc[b, a] = np.nan

  return correct_count

In [25]:
iters = 40 # amount of iterations to do
skips = 5 # amount of missing values to add at a time
bias = [15, 11, 5, 3, 2] # probability distribution across the items. higher = more often

res_biased = iter_test(iita_fm.mini_iita_fit, skips, iters, test_biased_items, bias=bias)

ITER 0
0
5
10
15
20
25
30
35
40
45
50
55
60
65
70
75
80
80
ITER 1
0
5
10
15
20
25
30
35
40
45
50
55
60
65
70
75
80
85
90
95
100
105
110
115
120
125
130
135
140
145
150
155
160
165
170
175
180
185
190
195
200
205
210
215
220
225
230
235
240
245
250
255
260
265
270
275
280
285
290
295
300
305
310
315
320
325
330
330
ITER 2
0
5
10
15
20
25
30
35
40
45
50
55
60
65
70
75
80
85
90
95
100
105
110
115
120
125
130
135
140
145
150
155
160
165
170
175
180
185
190
195
200
205
210
215
220
220
ITER 3
0
5
10
15
20
25
30
35
40
45
50
55
60
65
70
75
80
85
90
95
100
105
110
115
120
125
130
135
140
145
150
155
160
165
170
170
ITER 4
0
5
10
15
20
25
30
35
40
45
50
55
60
65
70
75
80
85
90
95
100
105
110
115
120
125
130
135
140
145
150
155
160
165
170
175
180
185
190
195
200
205
210
215
220
225
230
235
240
245
250
255
260
265
270
275
280
285
290
295
300
305
310
315
320
320
ITER 5
0
5
10
15
20
25
30
35
40
45
50
55
60
65
70
75
80
85
90
95
100
105
110
115
115
ITER 6
0
5
10
15
20
25
30
35
40
45
50
55
60
65
70
75

Running the tests is quite a long process, so I did it for 40 iterations and saved the results in the next cell

Skip the next cell if you run the tests on your own

In [29]:
res_biased = [
    80, 330, 220, 170, 320, 115, 200, 300, 260, 180,
    265, 220, 300, 235, 500, 175, 165, 150, 285, 385,
    210, 235, 165, 240, 145, 135, 230, 275, 175, 195,
    265, 155, 225, 230, 140, 135, 175, 330, 655, 345,
]

In [30]:
res_biased = (np.array(res_biased) / (data.shape[0] * data.shape[1])).round(3)

In [31]:
np.mean(res_biased).round(3)

np.float64(0.14)

In [32]:
np.std(res_biased).round(3)

np.float64(0.062)